# FinBERT Example Notebook

This notebooks shows how to train and use the FinBERT pre-trained language model for financial sentiment analysis.

## Modules 

In [1]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

/home/fangye/miniconda3/envs/finbert/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/fangye/miniconda3/envs/finbert/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; fo

In [2]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

### Setting path variables:
1. `lm_path`: the path for the pre-trained language model (If vanilla Bert is used then no need to set this one).
2. `cl_path`: the path where the classification model is saved.
3. `cl_data_path`: the path of the directory that contains the data files of `train.csv`, `validation.csv`, `test.csv`.
---

In the initialization of `bertmodel`, we can either use the original pre-trained weights from Google by giving `bm = 'bert-base-uncased`, or our further pre-trained language model by `bm = lm_path`


---
All of the configurations with the model is controlled with the `config` variable. 

In [3]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
cl_data_path = project_dir/'data'/'sentiment_data'

###  Configuring training parameters

You can find the explanations of the training parameters in the class docsctrings. 

In [5]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

Some weights of the model checkpoint at /home/fangye/finBERT/models/language_model/finbertTRC2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not ini

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [6]:
finbert = FinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [7]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

11/27/2023 20:28:05 - INFO - finbert.finbert -   device: cuda n_gpu: 8, distributed training: False, 16-bits training: False
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f35dc1a7750>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: a6a998e3-4e6f-40ce-9d3d-865e91dc73f5)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
11/27/2023 20:28:49 - WARNING - huggingface_hub.utils._http -   '(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f35dc1a7750>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: a

## Fine-tune the model

In [8]:
# Get the training examples
train_data = finbert.get_data('train')

In [10]:
train_data

 ...]

In [9]:
model = finbert.create_the_model()

/home/fangye/miniconda3/envs/finbert/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


### [Optional] Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. You can skip this part if you want to fine-tune the whole model.

<span style="color:red">Important: </span>
Execute this step if you want a shorter training time in the expense of accuracy.

In [11]:
# This is for fine-tuning a subset of the model.

freeze = 6

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

### Training

In [12]:
trained_model = finbert.train(train_examples = train_data, model = model)

11/27/2023 20:29:43 - INFO - finbert.utils -   *** Example ***
11/27/2023 20:29:43 - INFO - finbert.utils -   guid: train-1
11/27/2023 20:29:43 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
11/27/2023 20:29:43 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:29:43 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:29:43 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/

11/27/2023 20:29:43 - INFO - finbert.finbert -   ***** Loading data *****
11/27/2023 20:29:43 - INFO - finbert.finbert -     Num examples = 3488
11/27/2023 20:29:43 - INFO - finbert.finbert -     Batch size = 32
11/27/2023 20:29:43 - INFO - finbert.finbert -     Num steps = 48
Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/109 [00:00<?, ?it/s]

11/27/2023 20:29:47 - INFO - finbert.utils -   *** Example ***
11/27/2023 20:29:47 - INFO - finbert.utils -   guid: validation-1
11/27/2023 20:29:47 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
11/27/2023 20:29:47 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:29:47 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:29:47 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:29:47 - INFO - finbert.utils -   label: neutral (id = 2)
11/27/2023 20:29:47 - INFO - finbert.finbert -   ***** Loading data *****
11/27/2

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/109 [00:00<?, ?it/s]

11/27/2023 20:30:14 - INFO - finbert.utils -   *** Example ***
11/27/2023 20:30:14 - INFO - finbert.utils -   guid: validation-1
11/27/2023 20:30:14 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
11/27/2023 20:30:14 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:30:14 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:30:14 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:30:14 - INFO - finbert.utils -   label: neutral (id = 2)
11/27/2023 20:30:14 - INFO - finbert.finbert -   ***** Loading data *****
11/27/2

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Validation losses: [0.6579663799359248, 0.3691242474776048, 0.3389681291121703, 0.3208344807991615]


Epoch: 100%|██████████| 4/4 [00:31<00:00,  7.94s/it]


## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [13]:
test_data = finbert.get_data('test')

In [14]:
results = finbert.evaluate(examples=test_data, model=trained_model)

11/27/2023 20:30:41 - INFO - finbert.utils -   *** Example ***
11/27/2023 20:30:41 - INFO - finbert.utils -   guid: test-1
11/27/2023 20:30:41 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
11/27/2023 20:30:41 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:30:41 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:30:41 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:30:41 - IN

11/27/2023 20:30:41 - INFO - finbert.finbert -   ***** Loading data *****
11/27/2023 20:30:41 - INFO - finbert.finbert -     Num examples = 970
11/27/2023 20:30:41 - INFO - finbert.finbert -     Batch size = 32
11/27/2023 20:30:41 - INFO - finbert.finbert -     Num steps = 120
11/27/2023 20:30:41 - INFO - finbert.finbert -   ***** Running evaluation ***** 
11/27/2023 20:30:41 - INFO - finbert.finbert -     Num examples = 970
11/27/2023 20:30:41 - INFO - finbert.finbert -     Batch size = 32


Testing:   0%|          | 0/31 [00:00<?, ?it/s]

### Prepare the classification report

In [15]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [16]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [17]:
report(results,cols=['labels','prediction','predictions'])

Loss:0.40
Accuracy:0.83

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.81      0.78       267
           1       0.75      0.91      0.82       128
           2       0.90      0.82      0.86       575

    accuracy                           0.83       970
   macro avg       0.80      0.85      0.82       970
weighted avg       0.84      0.83      0.83       970



/home/fangye/miniconda3/envs/finbert/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  after removing the cwd from sys.path.


### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [18]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [19]:
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [21]:
import nltk

In [20]:
import nltk
nltk.download('punkt')

KeyboardInterrupt: 

In [22]:
result = predict(text,model)

11/27/2023 20:36:44 - INFO - root -   Using device: cpu 
11/27/2023 20:36:44 - INFO - finbert.utils -   *** Example ***
11/27/2023 20:36:44 - INFO - finbert.utils -   guid: 0
11/27/2023 20:36:44 - INFO - finbert.utils -   tokens: [CLS] later that day apple said it was rev ##ising down its earnings expectations in the fourth quarter of 2018 , largely because of lower sales and signs of economic weakness in china . [SEP]
11/27/2023 20:36:44 - INFO - finbert.utils -   input_ids: 101 2101 2008 2154 6207 2056 2009 2001 7065 9355 2091 2049 16565 10908 1999 1996 2959 4284 1997 2760 1010 4321 2138 1997 2896 4341 1998 5751 1997 3171 11251 1999 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:36:44 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:36:44 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

11/27/2023 20:36:44 - INFO - root -   tensor([[-1.5625,  3.2271, -0.4237],
        [-1.7979,  2.2904, -0.1343],
        [-1.6648,  3.4648, -0.8444],
        [-0.5254,  2.2278, -1.8908],
        [-1.6598,  3.3227, -0.9778]])
11/27/2023 20:36:44 - INFO - finbert.utils -   *** Example ***
11/27/2023 20:36:44 - INFO - finbert.utils -   guid: 0
11/27/2023 20:36:44 - INFO - finbert.utils -   tokens: [CLS] yields on government bonds fell as investors fled to the traditional haven in a market storm . [SEP]
11/27/2023 20:36:44 - INFO - finbert.utils -   input_ids: 101 16189 2006 2231 9547 3062 2004 9387 6783 2000 1996 3151 4033 1999 1037 3006 4040 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:36:44 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:36:44 - INFO - finbert.utils -   token_type_ids: 

In [23]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,"Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China.","[0.008039789, 0.9668518, 0.025108486]",negative,-0.958812,0.051746
1,The news rapidly infected financial markets.,"[0.015169578, 0.9047559, 0.08007458]",negative,-0.889586,0.000000
2,Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened.,"[0.0058064396, 0.98100513, 0.013188502]",negative,-0.975199,0.500000
3,"The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground.","[0.05900698, 0.92592937, 0.01506369]",negative,-0.866922,0.000000
4,Asian stockmarkets closed down on January 3rd and European ones opened lower.,"[0.006719755, 0.9799898, 0.013290448]",negative,-0.973270,-0.051111
5,Yields on government bonds fell as investors fled to the traditional haven in a market storm.,"[0.010462252, 0.96195, 0.027587807]",negative,-0.951488,0.000000


In [24]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is -0.94.


Here is another example

In [25]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [26]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

11/27/2023 20:37:30 - INFO - root -   Using device: cpu 
11/27/2023 20:37:30 - INFO - finbert.utils -   *** Example ***
11/27/2023 20:37:30 - INFO - finbert.utils -   guid: 0
11/27/2023 20:37:30 - INFO - finbert.utils -   tokens: [CLS] shares in the spin - off of south african e - commerce group nas ##pers surged more than 25 % in the first minutes of their market debut in amsterdam on wednesday . [SEP]
11/27/2023 20:37:30 - INFO - finbert.utils -   input_ids: 101 6661 1999 1996 6714 1011 2125 1997 2148 3060 1041 1011 6236 2177 17235 7347 18852 2062 2084 2423 1003 1999 1996 2034 2781 1997 2037 3006 2834 1999 7598 2006 9317 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:37:30 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:37:30 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

11/27/2023 20:37:30 - INFO - root -   tensor([[ 2.2251, -1.8736, -0.7324],
        [-0.8614, -2.0225,  2.2849],
        [ 0.2074, -2.9642,  2.1743],
        [ 2.3320, -2.3056, -0.3987],
        [ 2.0170, -2.2064, -0.4049]])
11/27/2023 20:37:30 - INFO - finbert.utils -   *** Example ***
11/27/2023 20:37:30 - INFO - finbert.utils -   guid: 0
11/27/2023 20:37:30 - INFO - finbert.utils -   tokens: [CLS] euro ##ne ##xt had given an indicative price of 58 . 70 euros per share for pro ##sus , implying a market value of 95 . 3 billion euros ( $ 105 billion ) . [SEP]
11/27/2023 20:37:30 - INFO - finbert.utils -   input_ids: 101 9944 2638 18413 2018 2445 2019 24668 3976 1997 5388 1012 3963 19329 2566 3745 2005 4013 13203 1010 20242 1037 3006 3643 1997 5345 1012 1017 4551 19329 1006 1002 8746 4551 1007 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
11/27/2023 20:37:30 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [27]:
result2

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,Shares in the spin-off of South African e-commerce group Naspers surged more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.,"[0.9358561, 0.015529718, 0.048614096]",positive,0.920326,0.250000
1,"Bob van Dijk, CEO of Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019.","[0.04071293, 0.012748294, 0.94653875]",neutral,0.027965,0.000000
2,"REUTERS/Piroschka van de Wouw Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a 31% stake in Chinese tech titan Tencent.","[0.12208747, 0.005120196, 0.87279236]",neutral,0.116967,0.000000
3,"There is 'way more demand than is even available, so that’s good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg.","[0.93035954, 0.009006326, 0.060634147]",positive,0.921353,0.533333
4,'It’s going to be an interesting hour of trade after opening this morning.',"[0.9062861, 0.013276493, 0.08043731]",positive,0.893010,0.500000
5,"Euronext had given an indicative price of 58.70 euros per share for Prosus, implying a market value of 95.3 billion euros ($105 billion).","[0.07702918, 0.008079121, 0.91489166]",neutral,0.068950,0.000000
6,The shares jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT.,"[0.26696947, 0.014659807, 0.71837074]",neutral,0.252310,0.000000


In [28]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))

Average sentiment is 0.46.
